In [ ]:
import os
import pandas as pd
import numpy as np
import PIL.Image
import time
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

In [ ]:
menu='''
                   Đồ Án Cuối Kỳ
            Nhập Môn Trí Tuệ Nhân Tạo
   bengali.AI Handwritten Grapheme Classification
            Đoàn Công Hậu - 1824801040101
            Hà Trọng Phan - 1824801040082
'''
print(menu)

In [ ]:
DATA_FOLDER = '../input/input-kaggle/'
train_df = pd.read_csv(os.path.join(DATA_FOLDER, 'train.csv'))
train_df.head()
print(train_df)

In [ ]:
train_df.shape

In [ ]:
test_df = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
test_df.head()

In [ ]:
test_df.shape

In [ ]:
class_map_df = pd.read_csv(os.path.join(DATA_FOLDER, 'class_map.csv'))
class_map_df.head()

In [ ]:
class_map_df.shape

In [ ]:
sample_submission_df = pd.read_csv(os.path.join(DATA_FOLDER, 'sample_submission.csv'))
sample_submission_df.head()

In [ ]:
sample_submission_df.shape

In [ ]:
start_time = time.time()
train_0_df = pd.read_parquet(os.path.join(DATA_FOLDER,'train_image_data_0.parquet'))
print(f"`train_image_data_0` read in {round(time.time()-start_time,2)} sec.")  

In [ ]:
train_0_df.shape

In [ ]:
train_0_df.head()

In [ ]:
start_time = time.time()
train_1_df = pd.read_parquet(os.path.join(DATA_FOLDER,'train_image_data_1.parquet'))
print(f"`train_image_data_1` read in {round(time.time()-start_time,2)} sec.")  

In [ ]:
train_1_df.shape

In [ ]:
train_1_df.head()

In [ ]:
start_time = time.time()
test_0_df = pd.read_parquet(os.path.join(DATA_FOLDER,'test_image_data_0.parquet'))
print(f"`test_image_data_0` read in {round(time.time()-start_time,2)} sec.") 

In [ ]:
test_0_df.shape

In [ ]:
test_0_df.head()

In [ ]:
print(f"Train: unique grapheme roots: {train_df.grapheme_root.nunique()}")
print(f"Train: unique vowel diacritics: {train_df.vowel_diacritic.nunique()}")
print(f"Train: unique consonant diacritics: {train_df.consonant_diacritic.nunique()}")
print(f"Train: total unique elements: {train_df.grapheme_root.nunique() + train_df.vowel_diacritic.nunique() + train_df.consonant_diacritic.nunique()}")
print(f"Class map: unique elements: \n{class_map_df.component_type.value_counts()}")
print(f"Total combinations: {pd.DataFrame(train_df.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic'])).shape[0]}")

In [ ]:
cm_gr = class_map_df.loc[(class_map_df.component_type=='grapheme_root'), 'component'].values
cm_vd = class_map_df.loc[(class_map_df.component_type=='vowel_diacritic'), 'component'].values  
cm_cd = class_map_df.loc[(class_map_df.component_type=='consonant_diacritic'), 'component'].values   

print(f"grapheme root:\n{15*'-'}\n{cm_gr}\n\n vowel discritic:\n{18*'-'}\n{cm_vd}\n\n consonant diacritic:\n{20*'-'}\n {cm_cd}")

In [ ]:
#theo giỏi điều tra giá trị thường xuyên
def most_frequent_values(data):
    total = data.count()
    tt = pd.DataFrame(total)
    tt.columns = ['Total']
    items = []
    vals = []
    for col in data.columns:
        itm = data[col].value_counts().index[0]
        val = data[col].value_counts().values[0]
        items.append(itm)
        vals.append(val)
    tt['Most frequent item'] = items
    tt['Frequence'] = vals
    tt['Percent from total'] = np.round(vals / total * 100, 3)
    return(np.transpose(tt))

In [ ]:
#giá trị thường xuyên nhất
most_frequent_values(train_df)

In [ ]:
#giá trị kiểm tra thường xuyên nhất
most_frequent_values(test_df)

In [ ]:
#phân bố giá trị các lớp(class)
def plot_count(feature, title, df, size=1):
    '''
    Số lô của các lớp của đối tượng địa lý đã chọn; tính năng là một giá trị phân loại
     param: feature - tính năng mà chúng tôi trình bày sự phân bố của các lớp
     param: title - tiêu đề hiển thị trong cốt truyện
     param: df - dataframe
     param: kích thước - kích thước (từ 1 đến n), nhân với 4 - kích thước của ô
    '''
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
    g.set_title("Số lượng và tỷ lệ phần trăm của {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show() 

In [ ]:
plot_count('grapheme_root', 'grapheme_root (20 giá trị thường xuyên nhất đầu tiên - train)', train_df, size=4)

In [ ]:
#nguyên âm
plot_count('vowel_diacritic', 'vowel_diacritic (train)', train_df, size=3)

In [ ]:
#phụ âm
plot_count('consonant_diacritic', 'consonant_diacritic (train)', train_df, size=3)

In [ ]:
def display_image_from_data(data_df, size=6):
    '''
   Hiển thị hình ảnh grapheme từ dữ liệu mẫu
     param: data_df - mẫu dữ liệu
     param: size - sqrt (kích thước mẫu dữ liệu)
    '''
    plt.figure()
    fig, ax = plt.subplots(size,size,figsize=(12,12))
   #hiển thị hình ảnh để lừa chọn kích cở
    for i, index in enumerate(data_df.index):
        image_id = data_df.iloc[i]['image_id']
        flattened_image = data_df.iloc[i].drop('image_id').values.astype(np.uint8)
        unpacked_image = PIL.Image.fromarray(flattened_image.reshape(137, 236))

        ax[i//size, i%size].imshow(unpacked_image)
        ax[i//size, i%size].set_title(image_id)
        ax[i//size, i%size].axis('on')

In [ ]:
display_image_from_data(train_0_df.sample(36))

In [ ]:
#vd cho mẫu số 2 bộ hình train_1 vs số lượng là 20 
display_image_from_data(train_1_df.sample(25), size = 5)

In [ ]:
'''- áp dụng chức năng này,để hiển thị không phải các grapheme ngẫu nhiên,
    mà là cùng một grapheme.

    - Đối với điều này, em thực hiện việc lấy mẫu (dựa trên sự biến đổi của gốc grapheme, 
dấu phụ nguyên âm và dấu phụ âm, làm tham số cho hàm).'''


def display_writting_variety(data_df=train_0_df, grapheme_root=72, vowel_diacritic=0,\
                             consonant_diacritic=0, size=5):
    '''
    Hàm này nhận một tập hợp các dấu gốc grapheme, dấu phụ nguyên âm và dấu phụ âm
     và hiển thị một mẫu gồm 25 hình ảnh cho grapheme này
     param: data_df - tập dữ liệu được sử dụng làm nguồn dữ liệu
     param: grapheme_root - nhãn gốc grapheme
     param: nguyên âm_diacritic - nhãn dấu phụ nguyên âm
     param: consonant_diacritic - nhãn dấu phụ âm
     param: size - sqrt (số lượng hình ảnh sẽ hiển thị)
    '''
    sample_train_df = train_df.loc[(train_df.grapheme_root == grapheme_root) & \
                                  (train_df.vowel_diacritic == vowel_diacritic) & \
                                  (train_df.consonant_diacritic == consonant_diacritic)]
    print(f"total: {sample_train_df.shape}")
    sample_df = data_df.merge(sample_train_df.image_id, how='inner')
    print(f"total: {sample_df.shape}")
    gr = sample_train_df.iloc[0]['grapheme']
    cm_gr = class_map_df.loc[(class_map_df.component_type=='grapheme_root')& \
                             (class_map_df.label==grapheme_root), 'component'].values[0]
    cm_vd = class_map_df.loc[(class_map_df.component_type=='vowel_diacritic')& \
                             (class_map_df.label==vowel_diacritic), 'component'].values[0]    
    cm_cd = class_map_df.loc[(class_map_df.component_type=='consonant_diacritic')& \
                             (class_map_df.label==consonant_diacritic), 'component'].values[0]    
    
    print(f"grapheme: {gr}, grapheme root: {cm_gr}, vowel discritic: {cm_vd}, consonant diacritic: {cm_cd}")
    sample_df = sample_df.sample(size * size)
    display_image_from_data(sample_df, size=size)

In [ ]:
#áp dụng hàm cho một vài kết hợp của dấu gốc grapheme, dấu phụ nguyên âm và dấu phụ âm.
display_writting_variety(train_0_df,72,1,1,4)

In [ ]:
display_writting_variety(train_0_df,64,1,2,4)

In [ ]:
#áp dụng hàm cho một vài kết hợp của dấu gốc grapheme, dấu phụ nguyên âm và dấu phụ âm. dùng bộ hình train_1
display_writting_variety(train_1_df,13,0,0,4)

In [ ]:
display_writting_variety(train_1_df,23,3,2,4,)